# Lesson 2 - motor and scan

In this lesson, we'll work with a motor (for precise positioning) in [Bluesky](http://nsls-ii.github.io/bluesky/) and [related tools](http://nsls-ii.github.io/).

First, we'll connect with an EPICS motor (using [ophyd](http://nsls-ii.github.io/ophyd/)), and then use the Bluesky software to scan the motor (with the scaler from [lesson 1](lesson1.ipynb).

Load `ophyd` device support for the `EpicsMotor` and connect with one EPICS motor channel.  We have a synApps (v5.8) XXX-style IOC with the prefix `prj:`. It has a scaler, 16 soft channel motors, and some other support we'll ignore in this lesson.

In [2]:
from ophyd import EpicsMotor
m1 = EpicsMotor("prj:m1", name="m1")

Show the current value fo the motor (that's the `.RBV` field, in case you were interested).

In [3]:
m1.position

0.0

Connect the scaler (as was done in lesson 1).  Define some of the chaannel names and clear out others.

In [4]:
from ophyd.scaler import ScalerCH
scaler = ScalerCH("prj:scaler1", name="scaler")

# Since there are no detectors actually connected to this scaler,
# we can change names at our choice.  A real scaler will have
# detectors connected to specific channels and we should not modify
# these names without regard to how signals are physically connected.
scaler.channels.chan01.chname.put("clock")
scaler.channels.chan02.chname.put("I0")
scaler.channels.chan03.chname.put("scint")
scaler.channels.chan04.chname.put("")
scaler.channels.chan05.chname.put("")
scaler.channels.chan06.chname.put("")
scaler.channels.chan07.chname.put("")
scaler.channels.chan08.chname.put("")
scaler.channels.chan09.chname.put("")

Use only the channels with EPICS names, those are the *interesting* channels.

In [5]:
from APS_BlueSky_tools.devices import use_EPICS_scaler_channels

scaler.match_names()
use_EPICS_scaler_channels(scaler)
scaler.read()

OrderedDict([('clock', {'value': 16000000.0, 'timestamp': 1533135553.458647}),
             ('I0', {'value': 4.0, 'timestamp': 1533135553.458647}),
             ('scint', {'value': 8.0, 'timestamp': 1533135553.458647}),
             ('scaler_time', {'value': 1.6, 'timestamp': 1533135553.458647})])

Create a RunEngine but do not connect it with a data collection strategy.  That will come in the next lessons.

In [6]:
from bluesky import RunEngine
import bluesky.plans as bp
RE = RunEngine({})

Run a step scan using the motor and the scaler.

In [10]:
RE(bp.scan([scaler], m1, -1, 1, 5))

('2fb37108-0e57-4d70-95db-5d451ae4af46',)

Ah, yes.  Nothing to see here since we did not setup anything to receive the *documents* from the RunEngine.  Here's the basic callback from lesson 1.

In [8]:
def myCallback(key, doc):
    print(key, len(doc))
    for k, v in doc.items():
        print("\t", k, v)

Repeat the same scan but handle the document stream with `myCallback()`.

In [9]:
RE(bp.scan([scaler], m1, -1, 1, 5), myCallback)

start 14
	 uid e6682f39-0f4e-42a0-8efe-87b185e46e7a
	 time 1533136065.810937
	 plan_pattern_module bluesky.plan_patterns
	 plan_args {'detectors': ["ScalerCH(prefix='prj:scaler1', name='scaler', read_attrs=['channels', 'channels.chan01', 'channels.chan01.s', 'channels.chan02', 'channels.chan02.s', 'channels.chan03', 'channels.chan03.s', 'time'], configuration_attrs=['channels', 'channels.chan01', 'channels.chan01.chname', 'channels.chan01.preset', 'channels.chan01.gate', 'channels.chan02', 'channels.chan02.chname', 'channels.chan02.preset', 'channels.chan02.gate', 'channels.chan03', 'channels.chan03.chname', 'channels.chan03.preset', 'channels.chan03.gate', 'channels.chan04', 'channels.chan04.chname', 'channels.chan04.preset', 'channels.chan04.gate', 'channels.chan05', 'channels.chan05.chname', 'channels.chan05.preset', 'channels.chan05.gate', 'channels.chan06', 'channels.chan06.chname', 'channels.chan06.preset', 'channels.chan06.gate', 'channels.chan07', 'channels.chan07.chname', 'cha

event 7
	 descriptor 2d2646de-00ce-45ec-95e6-29d561de7c33
	 time 1533136072.3648005
	 data {'clock': 16000000.0, 'I0': 9.0, 'scint': 11.0, 'scaler_time': 1.6, 'm1': -0.5, 'm1_user_setpoint': -0.5}
	 timestamps {'clock': 1533136072.361659, 'I0': 1533136072.361659, 'scint': 1533136072.361659, 'scaler_time': 1533136072.361659, 'm1': 1533136070.638063, 'm1_user_setpoint': 1533136070.738179}
	 seq_num 2
	 uid 6fa8e95f-81c6-42c0-8cba-9d9aaad020d7
	 filled {}
event 7
	 descriptor 2d2646de-00ce-45ec-95e6-29d561de7c33
	 time 1533136074.7777252
	 data {'clock': 16000000.0, 'I0': 8.0, 'scint': 8.0, 'scaler_time': 1.6, 'm1': 0.0, 'm1_user_setpoint': 0.0}
	 timestamps {'clock': 1533136074.767317, 'I0': 1533136074.767317, 'scint': 1533136074.767317, 'scaler_time': 1533136074.767317, 'm1': 1533136073.046235, 'm1_user_setpoint': 1533136073.14629}
	 seq_num 3
	 uid edb53847-3370-4c9a-9a6f-06cb7fbbf14e
	 filled {}
event 7
	 descriptor 2d2646de-00ce-45ec-95e6-29d561de7c33
	 time 1533136077.2726357
	 data

('e6682f39-0f4e-42a0-8efe-87b185e46e7a',)

# Summary

We'll show this code as a python program:

```
#!/APSshare/anaconda3/BlueSky/bin/python

"lesson 2: motor"

from ophyd import EpicsMotor
from ophyd.scaler import ScalerCH
from bluesky import RunEngine
import bluesky.plans as bp
from APS_BlueSky_tools.devices import use_EPICS_scaler_channels


def myCallback(key, doc):
    print(key, len(doc))
    for k, v in doc.items():
        print("\t", k, v)
    print("~~~~~~~~~~~~~~~~~")


m1 = EpicsMotor("prj:m1", name="m1")
print(m1.position)

scaler = ScalerCH("prj:scaler1", name="scaler")


# Since there are no detectors actually connected to this scaler,
# we can change names at our choice.  A real scaler will have
# detectors connected to specific channels and we should not modify
# these names without regard to how signals are physically connected.
scaler.channels.chan01.chname.put("clock")
scaler.channels.chan02.chname.put("I0")
scaler.channels.chan03.chname.put("scint")
scaler.channels.chan04.chname.put("")
scaler.channels.chan05.chname.put("")
scaler.channels.chan06.chname.put("")
scaler.channels.chan07.chname.put("")
scaler.channels.chan08.chname.put("")
scaler.channels.chan09.chname.put("")


scaler.match_names()
use_EPICS_scaler_channels(scaler)
print(scaler.read())

RE = RunEngine({})

RE(bp.scan([scaler], m1, -1, 1, 5))

RE(bp.scan([scaler], m1, -1, 1, 5), myCallback)
```